# Een lineair programma oplossen in Python: studenten indelen bij opdrachten.

### Introductie
In dit techreport bespreken we hoe een  lineair programma (LP) opgelost kan worden met behulp van Python. Lineair programmeren is een methode die gebruikt kan worden om optimalisatieproblemen op te lossen waarbij een doelfuctie wordt geminimaliseerd onder bepaalde randvoorwaarden. 

Ter illustratie introduceren we een eenvoudig LP model om de groepsindeling van een project te maken. Voor dit project zijn er een aantal verschillende opdrachten en studenten geven een voorkeur voor een project aan. Op basis van deze voorkeuren worden de groepen ingedeeld zodanig dat zoveel mogelijk studenten bij het project van hun voorkeur zitten. 

Eerst wordt het LP-model kort gegeven en vervolgens laten we zien hoe dit in Python wordt opgelost met behulp van pulp. Daarna bespreken we ook een aantal mogelijke uitbreidingen. 

### Het LP model
Een LP model bestaat uit verschillende elementen: de parameters en beslissingsvariabelen, de doelfunctie en de randvoorwaarde. De parameters geven de input van het model weer.
<br>
$n = \text{aantal studenten}$
<br>
$m = \text{aantal opdrachten}$
<br>
$
    v_{ij}=
    \begin{cases}
      1, & \text{als student $i$ voorkeur heeft voor opdracht $j$,} \\
      0, & \text{anders,}
    \end{cases} \quad i=1,...,n;j=1,...,m.
$
<br>
$g_{max} = \text{maximale groepsgrootte}$ 
<br>
$g_{min} = \text{minimale groepsgrootte}$ 
<br><br>
De beslissing die gemaakt moet worden is welke student in welk groepje wordt ingedeeld. Hiervoor worden de beslissingsvariabelen geintroduceerd.
<br>
$
    x_{ij}=
    \begin{cases}
      1, & \text{als student $i$ wordt ingedeeld bij opdracht $j$,} \\
      0, & \text{anders,}
    \end{cases} \quad  i=1,...,n;j=1,...,m.
$
<br>
Met behulp van de doelfunctie kan aangegeven wordt wat er geoptimaliseerd moet worden. In dit geval willen we het aantal studenten dat bij een opdracht van hun voorkeur wordt ingedeeld maximaliseren.
<br>
$ \max \sum_{i=1}^n\sum_{j=1}^mv_{ij}x_{ij}$
<br>
De randvoorwaarden geven aan aan welke voorwaarde er voldaan moet worden. De eerste randvoorwaarde zorgt ervoor dat elke student in een groepje wordt ingedeeld. De tweede en derde randvoorwaarden zorgen ervoor dat alle groepjes niet minder dan het minimaal aantal studenten en niet meer dan het maximaal aantal studenten heeft. De laatste randvoorwaarde (ook wel tekenvoorwaarde) zorgt ervoor dat $x_{ij}$ alleen de waarden 0 en 1 kan aannemmen. 
<br>
$\sum_{j=1}^m x_{ij} = 1  \quad\quad\quad\quad i=1,...,n$
<br>
$\sum_{i=1}^n x_{ij} \ge g_{min} \quad\quad\quad j=1,...,m$
<br>
$\sum_{i=1}^n x_{ij} \le g_{max} \quad\quad\quad j=1,...,m$
<br>
$x_{ij} \in \{0,1\} \quad\quad\quad\quad i=1,...,n;j=1,...,m$ 

### LP in Python

We laten nu zien hoe het LP-model in Python opgelost kan worden. Er zijn verschillende packages die gebruikt kunnem worden. Wij gebruiken hiervoor het package Pulp. Daarnaast worden de packages Numpy en Pandas gebruikt om het model in te voeren en de output te presenteren. 

In [1]:
import pulp as plp
import numpy as np
import pandas as pd

**Parameters**<br>
Ter illustratie gebruiken we eenvoudig voorbeeld met 20 studenten en 5 opdrachten. Elke groep bestaat uit 4 tot 6 personen. De formulering van het uiteindelijke LP is algemeen en kan voor willekeurige instanties opgelost worden. 

In [2]:
## parameters

n = 20 # aantal studenten
m = 5 # aantal opdrachten
g_min = 4 # minimaal aantal studenten per opdracht
g_max = 6 # maximaal aantal studenten per opdracht
# nxm matrix die de voorkeur van studenten voor opdrachten aangeeft
v = np.array([[0,1,0,1,0],
              [1,1,0,0,0],
              [0,1,0,1,0],
              [0,0,0,1,1],
              [0,1,0,1,0],
              [1,1,0,0,0],
              [0,1,1,0,0],
              [0,0,0,1,1],
              [0,1,0,1,0],
              [1,1,0,0,0],
              [0,0,1,1,0],
              [0,0,0,1,1],
              [0,1,0,0,1],
              [0,0,1,1,0],
              [1,0,0,1,0],
              [1,0,0,0,1],
              [1,1,0,0,0],
              [1,1,0,0,0],
              [0,1,0,0,1],
              [0,1,0,1,0]])

**Model**<br>
We maken eerst het LP model aan. Hierbij kunnen we ook al aangeven of het een  minimalisatie (plp.LpMinimize) of maximalisatie (plp.LpMaximize) is. 

In [3]:
# maak het model aan
lp_model = plp.LpProblem(name="LPgroepsindeling",sense=plp.LpMaximize)

**Beslissingsvariabelen**<br>
Vervolgens worden beslissingsvariabelen aangemaakt in een dictionary. Hiervoor wordt de functie plp.LpVariable gebruikt. We maken de variabele x_ij aan. Hierbij voegen we meteen de tekenrestricties toe. 

Je kunt kiezen uit 3 soorten categoriën: plp.LpInteger, plp.LpBinary, plp.LpContinuous. De argumenten lowBound en upBound zijn optioneel. Voor een binaire variabele zijn die niet nodig omdat die alleen de waarde 0 en één kunnen aangeven, maar bij een continue variabele kun je op deze manier meteen de grenzen aangeven (bijvoorbeeld lowBound=0, upBound=None).


In [4]:
# Variabelen
x = {(i,j): plp.LpVariable(cat=plp.LpBinary, 
                           name=f"x_{i}_{j}")
     for i in range(n) for j in range(m)}

**Doelfunctie en randvoorwaarden**<br>
Vervolgens worden de doelfunctie en randvoorwaarden één voor een aan het model toegevoegd. Let op: het is belangrijk om altijd eerst de doelfunctie 
toe te voegen en dan de randvoorwaarden. Om een som te kunnen nemen gebruik je de functie plp.lpSum()

In [5]:
# doelfuntie (voeg altijd als eerste de doelfunctie toe)
lp_model += (
    plp.lpSum(v[i,j] * x[i,j] for i in range(n) for j in range(m)),
    "objective")

In [6]:
# Randvoorwaarden 
for i in range(n):
    lp_model += (
        plp.lpSum(x[i,j] for j in range(m)) == 1,
        f"constraint1_{i}")
    
for j in range(m):
    lp_model += (
        plp.lpSum(x[i,j] for i in range(n)) >= g_min,
        f"constraint2_{j}")
    
for j in range(m):
    lp_model += (
        plp.lpSum(x[i,j] for i in range(n)) <= g_max,
        f"constraint3_{j}")

**LP oplossen**<br>
Nu de doelfunctie en alle randvoorwaarden zijn toegevoegd aan het model kunnen we het model oplossen. Aan de output kun je zien of het model is opgelost. Een aantal mogelijke outputs zijn: 
- 0: niet opgelost;
- 1: optimaal;
- -1: onoplosbaar;
- -2: onbegrensd;
- -3: ongedefiniëerd.

In [7]:
# Model oplossen
lp_model.solve()

#Uitkomsten 
objvalue = plp.value(lp_model.objective)

varsvalue = {}
for i in lp_model.variables():
    varsvalue[i.name] = i.varValue

**Uitkomsten**<br>
De uitkomsten van het model zijn nu te vinden in een dictionary. Het kan lastig zijn om de resultaten hieruit te halen om er verder mee te rekenen. Het is een optie om de resultaten in een dataframe te zetten, vanaf daar kun je er makkelijker verder mee werken.

In [8]:
# Resultaten in een dataframe presenteren
df = pd.DataFrame.from_dict(varsvalue.items()).loc[lambda d:d.iloc[:,0].str.contains('x')]
df.columns = ['variable','value']
df[['x','i','j']] = df.variable.str.split('_',expand = True)

df['i'] = df['i'].astype('int')
df['j'] = df['j'].astype('int')

df = df.sort_values(['i','j'])

# matrix met alle waarden van x_ij        
values_x = np.full((n,m),0.0)
values_x[df['i'],df['j']] = df['value']

print('doelfunctiewaarde:',objvalue)
print('toewijzing:',values_x.round())

doelfunctiewaarde: 19.0
toewijzing: [[0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]]


Als je dit model echt wilt gaan gebruiken om groepsindelingen te maken in het natuurlijk handig om hier een functie van te maken waarbij de parameters als input worden gegeven en de uitkomst als output. Dit zullen we bij volgende voorbeelden in één keer doen. 

### Mogelijke uitbreidingen

We hebben nu met een eenvoudig model voor groepsindelingen laten zien hoe een LP in Python geïmplementeerd kan worden. Het is natuurlijk mogelijk om dit model op allerlei manieren uit te breiden. We bespreken er 3: 
- weging voor opdrachten meegeven;
- mogelijkheid om een opdracht niet te doen;
- voorkeuren voor groepsgenoten.

**Weging voor opdrachten meegeven**<br>
In het model dat we hebben geintroduceerd hebben we aangenomen dat voor elke opdracht aangegeven wordt of de student deze wel of niet kan doen. Echter, je kunt je ook voorstellen dat de student een top 3 van opdrachten samenstelt. Het model hoeft hier niet voor te veranderen, alleen de definitie van de volgende parameter is iets aangepast: <br>
$
    v_{ij}=\text{de waarde die student $i$ geeft aan opdracht $j$}\quad i=1,...,n;j=1,...,m.
$<br>
Hierbij geldt: hoe hoger de waarde, hoe groter de voorkeur. Door met de mogelijke waardes te spelen kun je verschillende situaties modelleren, bijvoorbeeld:
- Als je  liever hebt dat alle studenten in hun top 3 worden ingedeeld dan dat de helft bij hun eerste voorkeur wordt ingedeeld en de andere helft bij een opdracht buiten hun top 3 kun je bijvoorbeeld de 3de keus een waarde 10 geven, de 2de keus een waarde 12 en de eerste keus een waarde van 15.
- Je zou opdrachten kunnen uitsluiten door de waarde negatief te maken.

**Mogelijkheid om een opdracht niet te doen** <br> 
Het is ook mogelijk om ervoor te kiezen om niet alle opdrachten uiteindelijk te doen, bijvoorbeeld als je te veel opdrachten hebt voor het aantal studenten. Om dit je kunnen doen zijn er wel een aantal aanpassingen nodig aan het model. De volgende beslissingsvariabele wordt geïntroduceerd:<br>
$
    y_{j}=
    \begin{cases}
      1, & \text{als opdracht $j$ wordt uitgevoerd,} \\
      0, & \text{anders,}
    \end{cases} \quad  i=1,...,n;j=1,...,m.
$
Om ervoor te zorgen dat studenten alleen worden ingedeeld bij opdrachten die ook daadwerkelijk worden uitgevoerd moeten de randvoorwaarden worden aangepast. De doelfunctie blijft hetzelfde. <br>
<br>
$\sum_{j=1}^m x_{ij} = 1  \quad\quad\quad\quad i=1,...,n$
<br>
$\sum_{i=1}^n x_{ij} \ge g_{min}\cdot y_j \quad j=1,...,m$
<br>
$\sum_{i=1}^n x_{ij} \le g_{min}\cdot y_j \quad j=1,...,m$
<br>
$x_{ij},y_j \in \{0,1\} \quad\quad\quad i=1,...,n;j=1,...,m$ <br>
Met onderstaande functie kan dit model ook worden opgelost in Pulp. 

In [9]:
def Groepsindeling2(n,m,g_min,g_max,v):
    # Deze functie implementeerd een LP model om een groepsindeling te maken waarbij studenten
    # een voorkeur kunnen aangeven voor een opdracht en mogelijk niet alle opdrachten worden uitgevoerd.
    # n: aantal studenten
    # m: aantal opdrachten
    # g_min: minimaal aantal studenten per opdracht
    # g_max: maximaal aantal studenten per opdracht
    # v: nxm matrix die de voorkeur van studenten voor opdrachten aangeeft
    
    # maak het model aan
    lp_model = plp.LpProblem(name="LPgroepsindeling",sense=plp.LpMaximize)

    # Variabelen
    x = {(i,j): plp.LpVariable(cat=plp.LpBinary, 
                               name=f"x_{i}_{j}")
         for i in range(n) for j in range(m)}

    y = {j: plp.LpVariable(cat=plp.LpBinary, 
                               name=f"y_{j}")
         for j in range(m)}
    
    # doelfuntie 
    lp_model += (
        plp.lpSum(v[i,j] * x[i,j] for i in range(n) for j in range(m)))
    
    # Randvoorwaarden 
    for i in range(n):
        lp_model += (
            plp.lpSum(x[i,j] for j in range(m)) == 1)
        
    for j in range(m):
        lp_model += (
            plp.lpSum(x[i,j] for i in range(n)) >= g_min * y[j])
        
    for j in range(m):
        lp_model += (
            plp.lpSum(x[i,j] for i in range(n)) <= g_max * y[j])
    # Model oplossen
    lp_model.solve()
    
    #Uitkomsten 
    objvalue = plp.value(lp_model.objective)
    
    varsvalue = {}
    for i in lp_model.variables():
        varsvalue[i.name] = i.varValue
    
    return objvalue, varsvalue

In [10]:
# oplossen
objvalue, varsvalue = Groepsindeling2(n,m,g_min,g_max,v)

print('doelfunctiewaarde: ', objvalue)

doelfunctiewaarde:  20.0


We zien dat de doelfunctiewaarde nu iets hoger is omdat er geen studenten worden ingedeeld bij opdracht 3.

**Voorkeuren voor groepsgenoten**<br> 
Ten slotte bekijken we een uitbreiding waarbij studenten kunnen aangeven met wie ze graag samen willen werken. Hiervoor wordt het model wat meer aangepast. We gaan weer uit van het basis model om mee te starten. De volgende extra parameter wordt geïntroduceerd:
$
    s_{ik}=
    \begin{cases}
      1, & \text{als als student $i$ met student $k$ wil samenwerken,} \\
      0, & \text{anders,}
    \end{cases} \quad  i,j=1,...,n.
$
Er is ook een extra beslissingsvariabele nodig:
$
    z_{ijk}=
    \begin{cases}
      1, & \text{als student $i$ en $k$ samen aan opdracht $j$ werken,} \\
      0, & \text{anders,}
    \end{cases} \quad  i=1,...,n;j=1,...,m.
$

De waarde van deze variabele is afhankelijk van de keuze van $x_{ij}$, hiervoor voegen we de volgende randvoorwaarden toe:<br>
$ z_{ijk} \ge x_{ij}+x{kj}-1 \quad i,k=1,...,n;j=1,...,m $<br>
$ z_{ijk} \le x_{ij} \quad i,k=1,...,n;j=1,...,m $<br>
$ z_{ijk} \le x_{kj} \quad i,k=1,...,n;j=1,...,m $ <br>
De eerste extra randvoorwaarde zorgt ervoor dat de waarde van $z_{ijk}$ gelijk is aan 1 als student $i$ en $k$ allebei aan project $j$ werken en de laatste twee extra randvoorwaarden zorgen ervoor dat $z_{ijk}$ gelijk is aan 0 als of student $i$ of student $k$ (of allebei) niet aan project $j$ werken. 

Tenslotte moet ook de doelfunctie worden aangepast omdat we ervoor willen zorgen dat studenten zoveel mogelijk worden ingedeeld met medestudenten die ze als voorkeur hebben aangegeven: <br>
<br>
$ \max c_1\sum_{i=1}^n\sum_{j=1}^mv_{ij}x_{ij} + c_2\sum_{i=1}^n\sum_{k=1}^ns_{ik}\sum_{j=1}^mz_{ijk} $
<br>
Het eerste deel geeft aan hoeveel studenten ook daarwerkelijk bij een opdracht van hun voorkeur zitten en het tweede deel geeft aan hoeveel studenten zijn ingedeeld met medestudenten met wie ze graag samenwerken. De constanten $c_1$ en $c_2$ kunnen gebruikt worden om aan te geven welk deel het belangrijkst is. Moeten studenten bijvoorbeeld in eerste instantie bij de opdracht naar keuze worden ingedeeld en wordt daarna pas gekeken naar de samenwerking, dan kun je de waarde van $c_1$ wat hoger kiezen dan die van $c_2$. 

Met onderstaande functie kan dit model worden opgelost in pulp.

In [11]:
def Groepsindeling3(n,m,g_min,g_max,v,s,c1=1,c2=1):
    # Deze functie implementeerd een LP model om een groepsindeling te maken waarbij studenten
    # zowel een voorkeur kunnen aangeven voor een opdracht als met wie ze willen samenwerken.
    # n: aantal studenten
    # m: aantal opdrachten
    # g_min: minimaal aantal studenten per opdracht
    # g_max: maximaal aantal studenten per opdracht
    # v: nxm matrix die de voorkeur van studenten voor opdrachten aangeeft
    # s: nxn matrix die de voorkeur van studenten voor medestudenten aangeeft
    # c1: score voor hoe belangrijk voorkeur van studenten voor opdracht is (default: c1=1)
    # c1: score voor hoe belangrijk voorkeur van studenten voor medestudenten is (default: c2=1)
    
    # maak het model aan
    lp_model = plp.LpProblem(name="LPgroepsindeling",sense=plp.LpMaximize)

    # Variabelen
    x = {(i,j): plp.LpVariable(cat=plp.LpBinary, 
                               name=f"x_{i}_{j}")
         for i in range(n) for j in range(m)}

    z = {(i,j,k): plp.LpVariable(cat=plp.LpBinary, 
                               name=f"z_{i}_{j}_{k}")
         for i in range(n) for j in range(m) for k in range(n)}
    
    # doelfuntie 
    lp_model += (
        c1*plp.lpSum(v[i,j] * x[i,j] for i in range(n) for j in range(m)) + 
        c2*plp.lpSum(s[i,k] * plp.lpSum(z[i,j,k] for j in range(m)) for i in range(n) for k in range(n)))
    
    # Randvoorwaarden 
    for i in range(n):
        lp_model += (
            plp.lpSum(x[i,j] for j in range(m)) == 1)
        
    for j in range(m):
        lp_model += (
            plp.lpSum(x[i,j] for i in range(n)) >= g_min)
        
    for j in range(m):
        lp_model += (
            plp.lpSum(x[i,j] for i in range(n)) <= g_max)

    for i in range(n):
        for j in range(m):
            for k in range(n):
                lp_model += (
                    z[i,j,k]>=x[i,j]+x[k,j]-1)
                lp_model += (
                    z[i,j,k]<=x[i,j])
                lp_model += (
                    z[i,j,k]<=x[k,j])
                
    # Model oplossen
    lp_model.solve()
    
    #Uitkomsten 
    objvalue = plp.value(lp_model.objective)
    
    varsvalue = {}
    for i in lp_model.variables():
        varsvalue[i.name] = i.varValue
    
    return objvalue, varsvalue

In [12]:
# nieuwe parameters
s = np.array([[0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
              [1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
              [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
              [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
              [0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])

c1 = 1 # score voor hoe belangrijk voorkeur van studenten voor opdracht is  
c2 = 1 # score voor hoe belangrijk voorkeur van studenten voor medestudenten is 

# oplossen
objvalue, varsvalue = Groepsindeling3(n,m,g_min,g_max,v,s,c1,c2)

print('doelfunctiewaarde: ', objvalue)

doelfunctiewaarde:  26.0


Deze uitkomst is iets lastiger te vergelijken met het basis model omdat de doelfunctie nu ook is veranderd. Hiervoor moeten de uitkomsten wat verder bekeken worden.

### Conclusie

We hebben nu gezien hoe een LP model om een groepsindeling te maken eruit ziet en hoe dat in Python geïmplementeerd kan worden. We hebben naar een aantal uitbreidingen gekeken, maar er zijn natuurlijk nog veel meer variaties mogelijk!